# Multilayer Network - Visualization 1 - Plotting a 'stack of networks'
* Overall, to plot a 3D network, we plot a 3D Scatter Plot to position the nodes and then add lines (edges) between points (nodes)
* Here I use Kamada Kawai to position networks on 2D planes and then add a fixed value to 'Z' to place one above the other.

In [1]:
import networkx as nx 
import plotly.graph_objects as plot3D
import pandas as pd
import numpy as np

In [2]:
def getTracesWithNodeInfo(G,z,nodeCommunity=False):
    if not nodeCommunity:
        nodeCommunity=[1 for n in range(G.number_of_nodes()+1)]
        
    position_3d=nx.kamada_kawai_layout(G)
    for node in position_3d:
        position_3d[node]=np.append(position_3d[node],z)
        
    #we need to seperate the X,Y,Z coordinates for Plotly
    x_nodes = [position_3d[i][0] for i in range(G.number_of_nodes())]# x-coordinates of nodes
    y_nodes = [position_3d[i][1] for i in range(G.number_of_nodes())]# y-coordinates
    z_nodes = [position_3d[i][2] for i in range(G.number_of_nodes())]# z-coordinates
    #We also need a list of edges to include in the plot
    edge_list = G.edges()
    
    #we  need to create lists that contain the starting and ending coordinates of each edge.
    x_edges=[]
    y_edges=[]
    z_edges=[]
    #need to fill these with all of the coordiates
    for edge in edge_list:
        #format: [beginning,ending,None]
        x_coords = [position_3d[edge[0]][0],position_3d[edge[1]][0],None]
        x_edges += x_coords

        y_coords = [position_3d[edge[0]][1],position_3d[edge[1]][1],None]
        y_edges += y_coords

        z_coords = [position_3d[edge[0]][2],position_3d[edge[1]][2],None]
        z_edges += z_coords

    #create a trace for the edges
    trace_edges = plot3D.Scatter3d(x=x_edges,
                            y=y_edges,
                            z=z_edges,
                            mode='lines',
                            line=dict(color='black', width=2),
                            hoverinfo='none')
    
    #create a trace for the nodes
    trace_nodes = plot3D.Scatter3d(x=x_nodes,
                            y=y_nodes,
                            z=z_nodes,
                            mode='markers',
                            marker=dict(symbol='circle',
                                        size=10,
                                        color=nodeCommunity, #color the nodes according to their community
                                        colorscale='Viridis', #either green or mageneta
                                        line=dict(color='black', width=0.5)),
                            text=list(G.nodes()),
                            hoverinfo='text')
    return [trace_edges,trace_nodes]

def getLayout(plotName):
    #we need to set the axis for the plot 
    axis = dict(showbackground=False,
                showline=False,
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                showspikes=False,
                title='')

    #also need to create the layout for our plot
    layout = plot3D.Layout(title=plotName,
                    showlegend=False,
                    width=1024,
                    height=980,
                    scene=dict(xaxis=dict(axis),
                            yaxis=dict(axis),
                            zaxis=dict(axis),
                            ),
                    hovermode='closest')
    return layout

def plot3Dnetwork(G,name,nodeCommunity=False):
    data = getTracesWithNodeInfo(G,nodeCommunity)
    fig = plot3D.Figure(data=data, layout=getLayout(name))
    fig.show()
    

In [3]:
def verticalEdges(nodeList,traces):
    newTraces=[]
    for node in nodeList:
        x_edges=[]
        y_edges=[]
        z_edges=[]
        node = str(node)
        for i in range(1,len(traces),2):
            if node in traces[i]['text']:
                index=traces[i]['text'].index(node)
                x_edges.append(traces[i]['x'][index])
                y_edges.append(traces[i]['y'][index])
                z_edges.append(traces[i]['z'][index])

        trace_edges = plot3D.Scatter3d(x=x_edges,
                                y=y_edges,
                                z=z_edges,
                                mode='lines',
                                line=dict(color='black', width=2),
                                hoverinfo='none')     
        newTraces.append(trace_edges)
        
    return newTraces

###### Samples network and selected nodes

In [13]:
G1 = nx.complete_graph(10)
G2 = nx.dodecahedral_graph()
G3 = nx.barbell_graph(6,2)
G4 = nx.sedgewick_maze_graph()
listOfNetworks=[G1,G2,G3,G4]
#These are the nodes we want to draw vertical edges
nodes=[10,4,13,9,5]

In [15]:
traces=[]
layersHeight=0.75

#Add the networks one up on the other 
for i,G in enumerate(listOfNetworks):
    traces.extend(getTracesWithNodeInfo(G,layersHeight*i))
#Add the vertical edges
newTraces=verticalEdges(nodes,traces)
traces.extend(newTraces)

name="Multlayer"
fig = plot3D.Figure(data=traces, layout=getLayout(name))
fig.show()
fig.write_html("Multilayer Network - Visualization 1.html")